# Open AI modellek finomhangolása

Ez a jegyzetfüzet az Open AI [Finomhangolás](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokumentációjában található aktuális útmutatás alapján készült.

A finomhangolás javítja az alapmodellek teljesítményét az alkalmazásod számára azáltal, hogy további, az adott felhasználási esetre vagy helyzetre releváns adatokkal és kontextussal újratanítjuk azt. Érdemes megjegyezni, hogy a prompt tervezési technikák, mint a _few shot learning_ és a _retrieval augmented generation_ lehetővé teszik az alapértelmezett prompt releváns adatokkal való kiegészítését a minőség javítása érdekében. Ezek a megközelítések azonban korlátozottak a célzott alapmodell maximális tokenablakának mérete által.

A finomhangolással valójában magát a modellt tanítjuk újra a szükséges adatokkal (ami lehetővé teszi, hogy sokkal több példát használjunk, mint amennyi a maximális tokenablakba belefér) – és egy _egyedi_ verzióját telepítjük a modellnek, amelynek már nincs szüksége példákra az inferencia idején. Ez nemcsak a prompt tervezés hatékonyságát javítja (nagyobb rugalmasságot ad a tokenablak más célokra való használatában), hanem potenciálisan a költségeinket is csökkenti (azáltal, hogy kevesebb tokent kell elküldenünk a modellnek az inferencia során).

A finomhangolás 4 lépésből áll:
1. Készítsd elő a tanító adatokat és töltsd fel azokat.
1. Futtasd a tanítási feladatot, hogy kapj egy finomhangolt modellt.
1. Értékeld ki a finomhangolt modellt, és ismételd a minőség javítása érdekében.
1. Telepítsd a finomhangolt modellt az inferencia céljára, ha elégedett vagy.

Fontos megjegyezni, hogy nem minden alapmodell támogatja a finomhangolást – a legfrissebb információkért [ellenőrizd az OpenAI dokumentációját](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst). Korábban finomhangolt modellt is finomhangolhatsz. Ebben a bemutatóban a `gpt-35-turbo` modellt használjuk célmodellként a finomhangoláshoz.

---


### 1.1 lépés: Készítse elő az adatkészletét

Építsünk egy chatbotot, amely segít megérteni az elemek periódusos rendszerét azáltal, hogy egy limerikkel válaszol egy elemről feltett kérdésekre. Ebben az _egyszerű_ bemutatóban csak egy adatkészletet hozunk létre a modell betanításához néhány minta válasszal, amelyek bemutatják az adatok elvárt formátumát. Egy valós használati esetben sokkal több példát kell létrehozni az adatkészlethez. Lehet, hogy egy nyílt adatkészletet is használhat (az alkalmazási területének megfelelően), ha létezik ilyen, és átalakíthatja azt a finomhangoláshoz való használathoz.

Mivel a `gpt-35-turbo`-ra koncentrálunk, és egyetlen körös választ (chat befejezést) keresünk, példákat hozhatunk létre [ebben a javasolt formátumban](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), amely tükrözi az OpenAI chat befejezési követelményeit. Ha többkörös beszélgetési tartalomra számít, akkor a [többkörös példa formátumot](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) használná, amely tartalmaz egy `weight` paramétert, hogy jelezze, mely üzeneteket kell (vagy nem kell) használni a finomhangolási folyamatban.

Itt a bemutatóhoz az egyszerűbb, egykörös formátumot fogjuk használni. Az adatok [jsonl formátumban](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) vannak, egy rekord soronként, mindegyik JSON-formátumú objektumként ábrázolva. Az alábbi részlet 2 rekordot mutat mintaként – a teljes mintakészletért (10 példa) lásd a [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) fájlt, amelyet a finomhangolási bemutatóhoz használunk. **Megjegyzés:** Minden rekordot _egy sorban_ kell definiálni (nem szabad több sorra bontani, mint ahogy az egy formázott JSON fájlban szokásos).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Egy valós használati esetben sokkal nagyobb példakészletre lesz szükség a jó eredményekhez – a kompromisszum a válaszok minősége és a finomhangolás ideje/költsége között lesz. Mi egy kis készletet használunk, hogy gyorsan befejezhessük a finomhangolást, és bemutathassuk a folyamatot. Lásd [ezt az OpenAI Cookbook példát](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) egy összetettebb finomhangolási bemutatóért.


---

### 1.2 lépés Adatkészlet feltöltése

Töltsd fel az adatokat a Files API segítségével [ahogyan itt le van írva](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Ne feledd, hogy a kód futtatásához először a következő lépéseket kell elvégezned:
 - Telepítetted az `openai` Python csomagot (győződj meg róla, hogy a verzió >=0.28.0 a legújabb funkciókhoz)
 - Beállítottad az `OPENAI_API_KEY` környezeti változót az OpenAI API kulcsodra
További információért lásd a kurzushoz biztosított [Beállítási útmutatót](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst).

Most futtasd a kódot, hogy létrehozz egy feltöltésre alkalmas fájlt a helyi JSONL fájlodból.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### 2.1 lépés: Hozza létre a finomhangolási feladatot az SDK-val


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### 2.2. lépés: A munka állapotának ellenőrzése

Íme néhány dolog, amit a `client.fine_tuning.jobs` API-val tehetsz:
- `client.fine_tuning.jobs.list(limit=<n>)` - Az utolsó n finomhangolási munka listázása
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Egy adott finomhangolási munka részleteinek lekérése
- `client.fine_tuning.jobs.cancel(<job_id>)` - Egy finomhangolási munka megszakítása
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Legfeljebb n esemény listázása a munkából
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

A folyamat első lépése a _tanítófájl érvényesítése_, hogy megbizonyosodjunk arról, hogy az adatok megfelelő formátumban vannak.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### 2.3 lépés: Események követése a haladás nyomon követéséhez


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### 2.4 lépés: Állapot megtekintése az OpenAI irányítópulton


A státuszt megtekintheti az OpenAI weboldalán is, a platform _Finomhangolás_ szekciójának felfedezésével. Ez megmutatja az aktuális feladat állapotát, és lehetővé teszi a korábbi feladatvégrehajtások történetének nyomon követését is. Ebben a képernyőképen látható, hogy az előző futás sikertelen volt, míg a második futás sikeresen lezajlott. Kontextusként, ez akkor történt, amikor az első futás egy hibásan formázott rekordokat tartalmazó JSON fájlt használt – miután ezt javították, a második futás sikeresen befejeződött, és a modellt használatra elérhetővé tette. 

![Finomhangolási feladat állapota](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.hu.png)


A státuszüzeneteket és a metrikákat is megtekintheti, ha tovább görget a vizuális irányítópulton, ahogy az alább látható:

| Üzenetek | Metrikák |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.hu.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.hu.png)|


---

### 3.1 lépés: Azonosító lekérése és a finomhangolt modell tesztelése kódban


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### 3.2 lépés: Finomhangolt modell betöltése és tesztelése a Playgroundban

Most kétféleképpen tesztelheti a finomhangolt modellt. Először is meglátogathatja a Playgroundot, és a Modellek legördülő menüből kiválaszthatja az újonnan finomhangolt modelljét a listából. A másik lehetőség a Finomhangolás panelen megjelenő „Playground” opció használata (lásd a fenti képernyőképet), amely elindítja a következő _összehasonlító_ nézetet, amely az alapmodellt és a finomhangolt modell verzióit egymás mellett mutatja a gyors értékeléshez.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.hu.png)

Egyszerűen töltse ki a rendszerkörnyezetet, amelyet a tanító adatokban használt, és adja meg a tesztkérdését. Észre fogja venni, hogy mindkét oldalon frissül az azonos kontextus és kérdés. Futtassa le az összehasonlítást, és látni fogja a különbséget a kimenetek között. _Figyelje meg, hogy a finomhangolt modell hogyan jeleníti meg a választ az Ön példáiban megadott formátumban, míg az alapmodell egyszerűen követi a rendszerparancsot_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.hu.png)

Észre fogja venni, hogy az összehasonlítás a tokenek számát is megadja mindkét modell esetében, valamint a lekérdezéshez szükséges időt. **Ez a konkrét példa egyszerűsített, hogy bemutassa a folyamatot, de nem tükröz valós adatokat vagy helyzetet**. Észreveheti, hogy mindkét minta ugyanannyi tokent mutat (a rendszerkörnyezet és a felhasználói kérés azonos), miközben a finomhangolt modell több időt vesz igénybe a lekérdezéshez (egyedi modell).

Valós helyzetekben nem egy ilyen játékpéldát fog használni, hanem valós adatokon finomhangol (pl. termékkatalógus ügyfélszolgálathoz), ahol a válasz minősége sokkal nyilvánvalóbb lesz. _Ebben a_ kontextusban az alapmodelllel egyenértékű válaszminőség eléréséhez több egyedi prompttervezésre lesz szükség, ami növeli a tokenhasználatot és potenciálisan a lekérdezéshez szükséges feldolgozási időt. _Ha ki szeretné próbálni, nézze meg az OpenAI Cookbook finomhangolási példáit._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Jogi nyilatkozat**:
Ezt a dokumentumot az AI fordító szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével fordítottuk le. Bár a pontosságra törekszünk, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az anyanyelvén tekintendő hiteles forrásnak. Fontos információk esetén szakmai, emberi fordítást javaslunk. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy félreértelmezésekért.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
